# Brand Logo and Name Extraction Pipeline

## Import Libraries

In [11]:
import os
import cv2
from pathlib import Path

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

from skimage.metrics import structural_similarity as ssim

In [3]:
root_dir = Path.cwd().parent.parent
root_dir

PosixPath('/mnt/d/Projects_D/Brand_Extractor')

## Config

In [4]:
class Config:
    sample_fps = 1 # Sample sample_fps frame per second

In [9]:
CONFIG = Config()

## Data

In [5]:
data_dir = root_dir / 'data' / 'video_preprocessing' / 'videos'

In [8]:
videos = list(data_dir.glob('*.mp4'))
videos

[PosixPath('/mnt/d/Projects_D/Brand_Extractor/data/video_preprocessing/videos/tanishq_mishra_vlogs_lulu_mall.mp4'),
 PosixPath('/mnt/d/Projects_D/Brand_Extractor/data/video_preprocessing/videos/the_explorer_raj_dlf_mall.mp4'),
 PosixPath('/mnt/d/Projects_D/Brand_Extractor/data/video_preprocessing/videos/travel_with_chris_crazy_market_spree.mp4')]

## Pipeline Steps

The pipeline consists of the following steps:
1. Video processing (extracting frames)
2. Image processing (extracting brand logo and name)
3. Text recognition (extracting brand name)
4. Saving the extracted brand logo and name

## Video Processing

In [ ]:
class VideoProcessor:
    def __init__(self, video_path):
        self.video_path = video_path
        self.video_name = video_path.stem
        self.video = cv2.VideoCapture(str(video_path))
        self.fps = self.video.get(cv2.CAP_PROP_FPS)
        self.total_frames = int(self.video.get(cv2.CAP_PROP_FRAME_COUNT))
        self.frame_width = int(self.video.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.frame_shape = (self.frame_height, self.frame_width)
        self.sample_fps = CONFIG.sample_fps
        
        self.frames = []
        self.similar_frames = []
    
    def extract_frames(self, frame_interval=1):
        frames = []
        for i in range(0, self.total_frames, (int(self.fps) // self.sample_fps * frame_interval)):
            self.video.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = self.video.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
        self.frames = frames
    
    def find_similar_frames(self, threshold=0.9):
        similar_frames = []
        for i in tqdm(range(len(self.frames)-1)):
            frame1 = self.frames[i]
            frame2 = self.frames[i+1]
            
            frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_RGB2GRAY)
            frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_RGB2GRAY)

            (score, _) = ssim(frame1_gray, frame2_gray, full=True)
            if score > threshold:
                similar_frames.append((i, i+1))
                
        self.similar_frames = similar_frames
                
    
    